<a href="https://colab.research.google.com/github/mukeshgollen/NLPlay-with-transformers/blob/main/BERT_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.5MB 28.5MB/s 
     |████████████████████████████████| 3.3MB 38.7MB/s 
     |████████████████████████████████| 901kB 43.2MB/s 
     |████████████████████████████████| 266kB 31.1MB/s 
     |████████████████████████████████| 122kB 41.8MB/s 
     |████████████████████████████████| 245kB 43.7MB/s 


In [ ]:
import pandas as pd
from transformers import AutoTokenizer 
from transformers import AutoModelForSequenceClassification
from transformers import AdamW
from transformers import get_scheduler
import torch
from torch import nn ,optim
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np 
from nltk.tokenize import RegexpTokenizer


In [ ]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


**IMPORTING** **THE** **DATASET**

In [ ]:
cd /content/drive/MyDrive/archive

/content/drive/MyDrive/archive


In [ ]:
ls


 Cleaned_data.csv  'IMDB Dataset.csv'


In [ ]:
Data = pd.read_csv("IMDB Dataset.csv")

In [ ]:
tokenizer = RegexpTokenizer("[\w']+")
for i in range(len(Data['review'])):
  Data['review'][i] = tokenizer.tokenize(Data['review'][i])

In [ ]:
print(Data.head(10))

                                              review sentiment
0  [One, of, the, other, reviewers, has, mentione...  positive
1  [A, wonderful, little, production, br, br, The...  positive
2  [I, thought, this, was, a, wonderful, way, to,...  positive
3  [Basically, there's, a, family, where, a, litt...  negative
4  [Petter, Mattei's, Love, in, the, Time, of, Mo...  positive
5  [Probably, my, all, time, favorite, movie, a, ...  positive
6  [I, sure, would, like, to, see, a, resurrectio...  positive
7  [This, show, was, an, amazing, fresh, innovati...  negative
8  [Encouraged, by, the, positive, comments, abou...  negative
9  [If, you, like, original, gut, wrenching, laug...  positive


In [ ]:
def Sentiment_conversion(sentiment):
  if sentiment =='positive':
    return  1
  else : return 0 

In [ ]:
Data['sentiment'] = [Sentiment_conversion(sentiment) for sentiment in Data['sentiment']]

In [ ]:
from datasets import Dataset
Data = Dataset.from_pandas(Data)

 **TOKENIZING**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def tokenize_function(yoyo):
  return tokenizer(yoyo['review'],is_split_into_words=True ,padding ="max_length",truncation = True,max_length=150)
tokenized_Data = Data.map(tokenize_function,batched = True)

In [ ]:
print(tokenized_Data)

Dataset({
    features: ['attention_mask', 'input_ids', 'review', 'sentiment', 'token_type_ids'],
    num_rows: 50000
})


**Splitting the Dataset**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
tokenized_Data = tokenized_Data.remove_columns(['review'])
tokenized_Data = tokenized_Data.rename_column("sentiment","labels")

In [ ]:
tokenized_Data = tokenized_Data.train_test_split(test_size = 0.3,shuffle =True)
train_data = tokenized_Data['train']
test_data = tokenized_Data['test']
train_data.set_format('torch')
test_data.set_format('torch')


In [ ]:
print(train_data)

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    num_rows: 35000
})


In [ ]:
train_dataloader = DataLoader(train_data,shuffle=True,batch_size=16)
test_dataloader = DataLoader(test_data,shuffle=True,batch_size=16)

Do keep in mind of batch_size while training 

**MODEL**

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
optimizer = AdamW(model.parameters(),lr = 5e-5)

In [ ]:
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
device = torch.device('cuda')
model.to(device)
model.resize_token_embeddings(len(tokenizer))


Embedding(30522, 768, padding_idx=0)

In [ ]:
loss_method = nn.CrossEntropyLoss()

In [ ]:
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    losses_train =[]
    train_acc = 0
    for batch in train_dataloader:
        ip = batch['input_ids'].to(device)
        at = batch['attention_mask'].to(device)
        lb = batch['labels'].to(device)
        outputs = model(input_ids=ip ,attention_mask = at)
        loss = loss_method(outputs.logits,lb)
        losses_train.append(loss.item())
        loss.backward()

        optimizer.step()
    
        optimizer.zero_grad()
        progress_bar.update(1)
    epoch_loss = np.mean(losses_train)
    print('Epoch :'+str(i+1))
    print('loss = '+str(epoch_loss))

Epoch :50000
loss = 0.3044142373573627
Epoch :50000
loss = 0.17646294814820523


In [ ]:
from datasets import load_metric
metric = load_metric('accuracy')
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8984666666666666}

In [ ]:
model_name = 'classifier.pt'
torch.save(model,'/content/drive/MyDrive/archive/{model_name}')
